# Reduceret echelonform

In [13]:
import numpy as np
from sympy import Matrix, latex

def reduced_row_echelon_form(matrix):
    # Konverter matricen til et Sympy matrix objekt
    sympy_matrix = Matrix(matrix)

    # Konverter Sympy matrix til et NumPy array for numeriske operationer
    numpy_array = np.array(sympy_matrix.tolist(), dtype=float)

    # Anvend Gaussian elimination til at få reduceret row echelon form
    num_rows, num_cols = numpy_array.shape
    lead = 0
    for r in range(num_rows):
        if lead >= num_cols:
            break

        i = r
        while numpy_array[i, lead] == 0:
            i += 1
            if i == num_rows:
                i = r
                lead += 1
                if num_cols == lead:
                    break

        numpy_array[[i, r], :] = numpy_array[[r, i], :]

        if numpy_array[r, lead] != 0:
            numpy_array[r, :] /= numpy_array[r, lead]

        for i in range(num_rows):
            if i != r:
                numpy_array[i, :] -= numpy_array[i, lead] * numpy_array[r, :]

        lead += 1

    # Konverter NumPy array til Sympy matrix
    echelon_matrix = Matrix(numpy_array)

    # Konverter matricen til latex format
    echelon_latex = latex(echelon_matrix)

    return echelon_latex

# Matric som skal reduceres (hint: hvis du skal finde echelonform til et ligningssystem, så indsæt matricen du får EFTER at bruge np.stack)
matrix = [[1, 1, 2, 1, 0],
          [1, 1, 3, 2, 1],
          [2, 2, 5, 4, 1]]

echelon_form = reduced_row_echelon_form(matrix)
print(echelon_form)

\left[\begin{matrix}1.0 & 1.0 & 0 & 0 & -2.0\\0 & 0 & 1.0 & 0 & 1.0\\0 & 0 & 0 & 1.0 & 0\end{matrix}\right]
